# **Fine tuning RoBerta model for binary sentiment classification using "imdb dataset" and LORA fine tuning technique**

In [1]:
!pip install transformers torch evaluate peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ib

In [13]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
import pandas as pd
from peft import get_peft_model
from peft import LoraConfig , TaskType
import numpy as np
import evaluate

In [3]:
dataset = load_dataset('Imdb')
dataset

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [8]:
tokenizer=AutoTokenizer.from_pretrained('bert-base-uncased')
def tokenizer_function(dataset):
  return tokenizer(dataset['text'], padding='max_length', truncation=True)

tokenized_data=dataset.map(tokenizer_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [17]:
# Reduce dataset size :
train_dataset=tokenized_data['train'].shuffle(seed=42).select(range(5000))
test_dataset=tokenized_data['test'].shuffle(seed=42).select(range(1000))

### Injecting Lora to the peft :

In [11]:
lora_config= LoraConfig(
    task_type=TaskType.SEQ_CLS , r=1 , lora_alpha=1 , lora_dropout=0.1
)

In [ ]:
model= AutoModelForSequenceClassification.from_pretrained('FacebookAI/roberta-base', num_labels=2)
model=get_peft_model(model, lora_config)

metric=evaluate.load('accuracy')

def compute_metrics(eval_pred):
  logits,labels=eval_pred
  predictions=np.argmax(logits)
  return metric.compute(predictions=predictions, references=labels)

training_args=TrainingArguments(
    output_dir='_trainer_test',
    evaluation_strategy= 'epoch',
    num_train_epochs=3
)
trainer=Trainer(
    model = model,
    args = training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
